# Handling Missing Data

## What is missing data?

Not all missing data is equal. At the heart of the matter, there exists the need to distinguish between two types of missingness:

* **Unknown but existing data**: This is data that we know exists, however, due to sparse or incomplete sampling, we do not actually know the value of it. There is some value there, and it would be useful to try and apply some sort of missing data interpolation technique in order to discover it.

   For example, in 2013 *The New York Times* published [a survey of income mobility in the United States](http://www.nytimes.com/2013/07/22/business/in-climbing-income-ladder-location-matters.html). As it happens often in datasets which drill this deep (to a county level), there were several counties for which the newspaper could not trace data. Yet it would be possible, and easy, if it was truly necessary to do so, to interpolate reasonable values for these counties based on data from the surrounding ones, for instance, or based on data from other counties with similar demographic profiles. This is fundamentally speaking, data that *can* be filled by some means.

<!-- ![Map of the US by Income Mobility](location_matters.png "Map of the US by Income Mobility") -->
 
  
* **Data that doesn't exist**: data that does not exist at all, in any shape or form.

  For example, it would make no sense to ask the average household income for residents of an industrial park or other such location where no people actually live. It would not *really* make sense to use 0 as a [sentinal value](https://en.wikipedia.org/wiki/Sentinel_value) in this case, either, because the existance of such a number implies in the first place the existance of people for whom an average can be taken&mdash;otherwise in trying to compute an average you are making a [divide by zero error](https://en.wikipedia.org/wiki/Division_by_zero)! This is, fundamentally speaking, data that *cannot* be filled by any means.


This is an important distinction to keep in mind, and implementing it in some standard way significantly complicates the picture. It means that to ask the question "is this data entry filled?" one must actually consider three possible answers: "Yes", "No, but it can be", and "No, and it cannot be". There seem to be two dominant paradigms for handling this distinction:

* **Bitpatterns**: Embed sentinal values into the array itself. For instance for integer data one might take `0` or `-9999` to signal unknown but existant data. This requires no overhead but can be confusing and oftentimes robs you of values that you might otherwise want to use (like `0` or `-9999`).


* **Masks**: Use a seperate boolean array to "mask" the data whenever missing data needs to be represented. This requires making a second array and knowing when to apply it to the dataset, but is more robust.

[Numpy](http://www.numpy.org/) is the linear algebra and vectorized mathematical operation library which underpins the Python scientific programming stack, and its methodologies inform how everything else works. Numpy has masks: these are provided via the `numpy.ma` module. But it has no native bitpatterns! There is still no performant native bitpattern `NA` type available whatsoever.

The lack of a native `NA` type, as is the case in, say, R, is a **huge** problem for libraries, like [Pandas](http://pandas.pydata.org/), that should be able to efficiently handle large datasets.

Indeed, **Pandas does not use the `numpy.ma` mask**. Masks are simply non-performant above for the purposes of a library that is expected to be able to handle literally millions of entries entirely in-memory, as `pandas` does. Pandas instead defines and uses its own null value sentinels, particularly `NaN` (`np.nan`) for null numbers and `NaT` (a psuedo-native handled under-the-hood); and then allows you to apply your own `isnull()` mask to your dataset (more on that shortly). 


### ``None``: Pythonic missing data

The first sentinel value used by Pandas is ``None``, a Python singleton object that is often used for missing data in Python code.
Because it is a Python object, ``None`` cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type ``'object'`` (i.e., arrays of Python objects):

In [1]:
import numpy as np
import pandas as pd

In [2]:
val=np.array([6,None,7,5,6])
val

array([6, None, 7, 5, 6], dtype=object)

This ``dtype=object`` means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects.
While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types:

The use of Python objects in an array also means that if you perform aggregations like ``sum()`` or ``min()`` across an array with a ``None`` value, you will generally get an error:

### ``NaN``: Missing numerical data

The other missing data representation, ``NaN`` (acronym for *Not a Number*), is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [3]:
val1=np.array([1,np.nan,7,1,8])
val1

array([ 1., nan,  7.,  1.,  8.])

Notice that NumPy prefered a native floating-point type for this array: this means that unlike the object array from before, this array supports fast operations pushed into compiled code.
You should be aware that ``NaN`` is a bit like data virus–it infects any other object it touches.
Regardless of the operation, the result of arithmetic with ``NaN`` will be another ``NaN``:

In [4]:
6+np.nan

nan

In [5]:
7*np.nan

nan

NumPy does provide some special aggregations that will ignore these missing values:

In [6]:
np.nansum(val1)

17.0

Keep in mind that ``NaN`` is specifically a floating-point value; there is no equivalent NaN value for integers, strings, or other types.

### NaN and None in Pandas

``NaN`` and ``None`` both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [7]:
pd.Series([1,np.nan,2,None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

Notice that in addition to casting the integer array to floating point, Pandas automatically converts the ``None`` to a ``NaN`` value.
(Be aware that there is a proposal to add a native integer NA to Pandas in the future; as of this writing, it has not been included).

While this type of magic may feel a bit hackish compared to the more unified approach to NA values in domain-specific languages like R, the Pandas sentinel/casting approach works quite well in practice and in my experience only rarely causes issues.

The following table lists the upcasting conventions in Pandas when NA values are introduced:

|Typeclass     | Conversion When Storing NAs | NA Sentinel Value      |
|--------------|-----------------------------|------------------------|
| ``floating`` | No change                   | ``np.nan``             |
| ``object``   | No change                   | ``None`` or ``np.nan`` |
| ``integer``  | Cast to ``float64``         | ``np.nan``             |
| ``boolean``  | Cast to ``object``          | ``None`` or ``np.nan`` |

Keep in mind that in Pandas, string data is always stored with an ``object`` dtype.

## Operating on Null Values

As we have seen, Pandas treats ``None`` and ``NaN`` as essentially interchangeable for indicating missing or null values.
To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures.
They are:

- ``isnull()``: Generate a boolean mask indicating missing values
- ``notnull()``: Opposite of ``isnull()``
- ``dropna()``: Return a filtered version of the data
- ``fillna()``: Return a copy of the data with missing values filled or imputed

We will conclude this section with a brief exploration and demonstration of these routines.

### Create dataframe with missing values

In [10]:
row_data={'first_name':['Jason',np.nan,'Tina','Jake','Amy'],
         'last_name':['Miller',np.nan,'Ali','Milner','Cooze'],
         'age':[42,np.nan,36,24,73],
         'sex':['m',np.nan,'f','m','f'],
         'preTestScore':[4,np.nan,np.nan,2,3],
         'postTestScore':[25,np.nan,np.nan,62,70]}
print(row_data)
df=pd.DataFrame(raw_data,columns=['first_name','last_name','age','sex','preTestScore','postTestScore'])
df

{'first_name': ['Jason', nan, 'Tina', 'Jake', 'Amy'], 'last_name': ['Miller', nan, 'Ali', 'Milner', 'Cooze'], 'age': [42, nan, 36, 24, 73], 'sex': ['m', nan, 'f', 'm', 'f'], 'preTestScore': [4, nan, nan, 2, 3], 'postTestScore': [25, nan, nan, 62, 70]}


,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


### Drop missing observations

In [11]:
df_no_missing=df.dropna()
df_no_missing

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


### Drop rows where all cells in that row is NA

In [12]:
df_cleaned=df.dropna(how='all')
df_cleaned

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


### Create a new column full of missing values

In [15]:
df['location']=np.nan
df

,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42.0,m,4.0,25.0,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Tina,Ali,36.0,f,NaN,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0,NaN
4,Amy,Cooze,73.0,f,3.0,70.0,NaN


### Drop column if they only contain missing values

In [16]:
df.dropna(axis=1,how='all')

,first_name,last_name,age,sex,preTestScore,postTestScore
0,Jason,Miller,42.0,m,4.0,25.0
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Tina,Ali,36.0,f,NaN,NaN
3,Jake,Milner,24.0,m,2.0,62.0
4,Amy,Cooze,73.0,f,3.0,70.0


### Drop rows that contain less than five observations

This is mostly useful for time series

In [18]:
print(df)
df.dropna(thresh=5)

  first_name last_name   age  sex  preTestScore  postTestScore  location
0      Jason    Miller  42.0    m           4.0           25.0       NaN
1        NaN       NaN   NaN  NaN           NaN            NaN       NaN
2       Tina       Ali  36.0    f           NaN            NaN       NaN
3       Jake    Milner  24.0    m           2.0           62.0       NaN
4        Amy     Cooze  73.0    f           3.0           70.0       NaN


,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42.0,m,4.0,25.0,NaN
3,Jake,Milner,24.0,m,2.0,62.0,NaN
4,Amy,Cooze,73.0,f,3.0,70.0,NaN


### Fill in missing data with zeros

In [19]:
df.fillna(0)

,first_name,last_name,age,sex,preTestScore,postTestScore,location
0,Jason,Miller,42.0,m,4.0,25.0,0.0
1,0,0,0.0,0,0.0,0.0,0.0
2,Tina,Ali,36.0,f,0.0,0.0,0.0
3,Jake,Milner,24.0,m,2.0,62.0,0.0
4,Amy,Cooze,73.0,f,3.0,70.0,0.0


### Fill in missing in preTestScore with the mean value of preTestScore

inplace=True means that the changes are saved to the df right away

In [27]:
df["preTestScore"].fillna(df["preTestScore"].mean(),inplace=True)
df

TypeError: unsupported operand type(s) for +: 'float' and 'method'